In [1]:
from db_management_libs import *
from libs import *
from os import mkdir, getcwd
from selenium import webdriver

import re
import time

In [2]:
user_account = {
    "email": "automail.robort@yandex.com",
    "pass": "fGwk093oiVtdJPuc8z2wtFjGcvzSi6eonkvWTjZa8glIE6uYJdvLTJv9dJWJVDgMy68Bozma3kKGkWDw"
}

In [3]:
def init_folder(name):
    path = f'{getcwd()}/{name}'
    try:
        mkdir(path)
    finally:
        return path

temp = init_folder('temp_00')
users_folder = init_folder('storage')
users_images = init_folder('images')

In [4]:
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 1 
})

In [5]:
driver = webdriver.Chrome(chrome_options=options, executable_path='/home/shanenoi/Desktop/Selenium/WebDriver/chromedriver')
# driver = webdriver.Chrome('/home/shanenoi/Desktop/Selenium/WebDriver/chromedriver')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [6]:
web_ctrl = WebControl(driver)

In [7]:
# Login
web_ctrl.driver.get('https://facebook.com')

mail_holder = web_ctrl.driver.find_element_by_id('email')
mail_holder.clear()
mail_holder.send_keys(user_account['email'])

pass_holder = web_ctrl.driver.find_element_by_id('pass')
pass_holder.clear()
pass_holder.send_keys(user_account['pass'])

web_ctrl.driver.find_element_by_id('u_0_b').click()

In [8]:
current_task = CurrentTask(f'{temp}/user')

In [9]:
sql = SQL
DB_ARCHITECTURE = {
    "images": {
        "images": [
            sql.TYPE_TEXT
        ],
        "note": [
            sql.TYPE_TEXT
        ]
    },
    "friends": {
        "friends": [
            sql.TYPE_TEXT
        ],
    }
    
}

In [10]:
class Pers(object):
    
    def __init__(self, web_control, user_id):
        self.user_db = DataBase(f'{users_folder}/{user_id}.db')
        self.user_id = user_id
        self.web_ctrl = web_control
        self.web_ctrl.driver.get(f'https://facebook.com/{user_id}')
    
    def get_user_name(self):
        return self.web_ctrl.driver.execute_script(
            CrawlingJs\
            .convert_jsScript(
                CrawlingJs\
                .crawling_by_js['user_name']
            )
        )
    
    def load_more(self):
        self.web_ctrl.driver.execute_script(
            CrawlingJs\
            .convert_jsScript(
                CrawlingJs\
                .crawling_by_js['scroll_to_the_end_of_page']
            )
        )
        
    @staticmethod
    def get_image_name_and_size(url: str) -> list:
        full_url = re.findall('(https://[^()]+)', url)[0]
        result = re.findall('(\d+x\d+)|(\w+.jpg)',full_url)
        return [
            f'{result[0][0]}__{result[1][1]}',
            full_url
        ]
    
    @staticmethod
    def get_friend_name(url: str) -> str:
        return re.findall(
            'https://www.facebook.com/((profile\.php\?id=\d+)|([^/?]+))',
            url
        )[0][0]
        
    def get_list_friends(self):
        return map(
            self.get_friend_name,
            self.web_ctrl.driver.execute_script(
                CrawlingJs\
                .convert_jsScript(
                    CrawlingJs\
                    .crawling_by_js['list_friends']
                )
            )
        )
    
    def get_list_images(self):
        return map(
            self.download_image,
            self.web_ctrl.driver.execute_script(
                CrawlingJs\
                .convert_jsScript(
                    CrawlingJs\
                    .crawling_by_js['list_images']
                )
            )
        )
    
    def download_image(self, url):
        values = self.get_image_name_and_size(url)
        internal_path = f'{users_images}/{values[0]}'
        if not current_task.isdone(values[0]):
            with open(internal_path, 'wb') as imf:
                imf.write(self.web_ctrl.get_content(values[1]))
            current_task.add(values[0])
        else:
            print(f"[+] {values[0]} was downloaded!")
        return internal_path
    
    def create_userdb(self):
        if not current_task.isdone(self.user_id):
            for table in DB_ARCHITECTURE.keys():
                self.user_db.create_table(
                    name=table,
                    architecture=DB_ARCHITECTURE[table]
                )
            current_task.add(self.user_id)
        else:
            print(f"[+] {self.user_id} was created!")
        # because this function will return generator
        list(self.user_db.run())
            
    def save(self, n_friends:int = 0, n_images: int = 0):
        """
        [+] n_friends: the number of scrolling to the bottom of friends page
        [+] n_images: the number of scrolling to the bottom of images page
        """
        self.web_ctrl.driver.execute_script(
                CrawlingJs\
                .convert_jsScript(
                    CrawlingJs\
                    .crawling_by_js['friends']
                )
        )
        current = self.web_ctrl.driver.execute_script(
                CrawlingJs\
                .convert_jsScript(
                    CrawlingJs\
                    .crawling_by_js['length_page']
                )
        )
        for i in range(n_friends):
            while (self.web_ctrl.driver.execute_script(
                    CrawlingJs\
                    .convert_jsScript(
                        CrawlingJs\
                        .crawling_by_js['length_page']
                    )
            ) == current):
                self.web_ctrl.driver.execute_script(
                    CrawlingJs\
                    .convert_jsScript(
                        CrawlingJs\
                        .crawling_by_js['scroll_to_the_end_of_page']
                    )
                )
                current = self.web_ctrl.driver.execute_script(
                    CrawlingJs\
                    .convert_jsScript(
                        CrawlingJs\
                        .crawling_by_js['length_page']
                    )
                )
        friends = list(self.get_list_friends())
        self.web_ctrl.driver.execute_script(
            "console.log('hello');"
        )
        time.sleep(3)
        
        self.web_ctrl.driver.execute_script(
                CrawlingJs\
                .convert_jsScript(
                    CrawlingJs\
                    .crawling_by_js['images']
                )
        )
        current = self.web_ctrl.driver.execute_script(
                CrawlingJs\
                .convert_jsScript(
                    CrawlingJs\
                    .crawling_by_js['length_page']
                )
        )
        for i in range(n_images):
            while (self.web_ctrl.driver.execute_script(
                    CrawlingJs\
                    .convert_jsScript(
                        CrawlingJs\
                        .crawling_by_js['length_page']
                    )
            ) == current):
                self.web_ctrl.driver.execute_script(
                    CrawlingJs\
                    .convert_jsScript(
                        CrawlingJs\
                        .crawling_by_js['scroll_to_the_end_of_page']
                    )
                )
                current = self.web_ctrl.driver.execute_script(
                    CrawlingJs\
                    .convert_jsScript(
                        CrawlingJs\
                        .crawling_by_js['length_page']
                    )
                )
        images = list(self.get_list_images())
        time.sleep(3)

        while images or friends:
            if images:
                self.user_db.insert_data(
                    table="images",
                    specify_columns=['images'],
                    values=[images.pop(0)]
                )
            if friends:
                self.user_db.insert_data(
                    table="friends",
                    specify_columns=['friends'],
                    values=[friends.pop(0)]
                )
        list(self.user_db.run())

In [11]:
p = Pers(web_ctrl, 'zuck')
p.create_userdb()

['CREATE TABLE images (images TEXT, note TEXT)', '']
['CREATE TABLE friends (friends TEXT)', '']


In [12]:
p.save(
    n_friends=6,
    n_images=6
)

['INSERT INTO images (images) VALUES(?)', ['/home/shanenoi/Desktop/Hot-Girl-Detection/images/206x206__93883138_10111823689493271_3710543854718418944_o.jpg']]
['INSERT INTO friends (friends) VALUES(?)', ['profile.php?id=100046178985144']]
['INSERT INTO images (images) VALUES(?)', ['/home/shanenoi/Desktop/Hot-Girl-Detection/images/206x206__92468945_10111803329479901_757229135448768512_n.jpg']]
['INSERT INTO friends (friends) VALUES(?)', ['sajib.mina.56']]
['INSERT INTO images (images) VALUES(?)', ['/home/shanenoi/Desktop/Hot-Girl-Detection/images/206x206__91196513_10111748657003981_3422175455497682944_o.jpg']]
['INSERT INTO friends (friends) VALUES(?)', ['rhenz.cruz.7146']]
['INSERT INTO images (images) VALUES(?)', ['/home/shanenoi/Desktop/Hot-Girl-Detection/images/206x206__90521698_10111694356288051_5219671938057633792_o.jpg']]
['INSERT INTO friends (friends) VALUES(?)', ['ravan.monsters.942']]
['INSERT INTO images (images) VALUES(?)', ['/home/shanenoi/Desktop/Hot-Girl-Detection/images/